# Orbit Propagation Demo 

This tutorial demonstrates how to perform orbit propagation through numerical integration.

## Imports

In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from ostk.astrodynamics.trajectory.state import NumericalSolver
from ostk.astrodynamics.display import create_2d_map, create_3d_globe
from ostk.astrodynamics.flight.system import SatelliteSystem, PropulsionSystem
from ostk.astrodynamics.trajectory import LocalOrbitalFrameFactory
from ostk.astrodynamics.trajectory import LocalOrbitalFrameDirection
from ostk.astrodynamics.trajectory.state import CoordinatesSubset
from ostk.astrodynamics.trajectory.state.coordinates_subset import CartesianPosition
from ostk.astrodynamics.trajectory.state.coordinates_subset import CartesianVelocity
from ostk.astrodynamics.trajectory.state import CoordinatesBroker
from ostk.astrodynamics.dynamics import (
    AtmosphericDrag,
    CentralBodyGravity,
    PositionDerivative,
    ThirdBodyGravity,
)
from ostk.astrodynamics.dynamics.thruster import ConstantThrustThruster
from ostk.astrodynamics.trajectory import Orbit, Propagator, State
from ostk.astrodynamics.trajectory.orbit.models.kepler import COE
from ostk.core.filesystem import Directory
from ostk.mathematics.geometry.d3.objects import Composite, Cuboid, Point
from ostk.physics import Environment
from ostk.physics.coordinate import Frame, Position, Velocity
from ostk.physics.coordinate.spherical import LLA
from ostk.physics.environment.atmospheric import Earth as EarthAtmosphericModel
from ostk.physics.environment.gravitational import Earth as EarthGravitationalModel
from ostk.physics.environment.magnetic import Earth as EarthMagneticModel
from ostk.physics.environment.objects.celestial_bodies import Earth, Moon, Sun
from ostk.physics.time import DateTime, Duration, Instant, Interval, Scale
from ostk.physics.units import Mass

ImportError: cannot import name 'ConstantThrustThruster' from 'ostk.astrodynamics.dynamics.thruster' (unknown location)

## Simulation Setup

### **Define the physical `Environment`**

In [2]:
instant_J2000 = Instant.J2000()
celestial_objects = [
    Earth.EGM96(360, 360),  # Earth.EGM2008(), Earth.WGS84_EGM96(), Earth.EGM84(180, 180)
    Sun.default(),
    Moon.default(),
]

environment = Environment(instant_J2000, celestial_objects)

### **Define the `PropulsionSystem` with properties:**
* `Thrust` (Newton)
* `SpecificImpusle` (s)

In [3]:
propulsion_system = PropulsionSystem(
    1.0,
    150.0,
)

### **Define the Thrust Direction in Local Orbital Frame:**

In [4]:
thrust_direction = LocalOrbitalFrameDirection(
    [1.0, 0.0, 0.0],
    LocalOrbitalFrameFactory.VNC(Frame.GCRF()),
)

### **Define the `SatelliteSystem` with properties:**
* `Mass`
* `Geometry`
* `Inertia Tensor`
* `Surface Area`
* `Drag Coefficient`

In [5]:
mass = Mass(90.0, Mass.Unit.Kilogram)
satellite_geometry = Composite(
    Cuboid(
        Point(0.0, 0.0, 0.0),
        [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]],
        [1.0, 0.0, 0.0],
    )
)
inertia_tensor = np.identity(3)
surface_area = 0.8
drag_coefficient = 2.2

satellite_system = SatelliteSystem(
    mass,
    satellite_geometry,
    inertia_tensor,
    surface_area,
    drag_coefficient,
    propulsion_system,
)

### **Define the initial `State` of the `SatelliteSystem`**:

In [6]:
coordinates_broker = CoordinatesBroker(
    [
        CartesianPosition.default(),
        CartesianVelocity.default(),
        CoordinatesSubset.mass(),
    ]
)

start_instant = Instant.date_time(DateTime.parse("2023-03-20T00:00:00.000"), Scale.UTC)
propellant_mass_kilograms = 15.0
coordinates = [
    -1514668.9408102570269,
    -192084.12149140036718,
    6831711.4584368728174,
    -6348.0791876050259859,
    3867.5824926981121621,
    -1297.1761044290490705,
    satellite_system.get_mass().in_kilograms() + propellant_mass_kilograms,
]

start_state = State(start_instant, coordinates, Frame.GCRF(), coordinates_broker)

### **Define the Dynamics to consider for Propagation**
* Central Body Gravity
* Third Body Gravity
* Atmospheric Drag
* Thruster Dynamics (Constant Thrust through propagation)

In [7]:
earth = Earth.from_models(
    EarthGravitationalModel(
        EarthGravitationalModel.Type.EGM2008,
        Directory.undefined(),
        20,
        20,
    ),
    EarthMagneticModel(EarthMagneticModel.Type.Undefined),
    EarthAtmosphericModel(EarthAtmosphericModel.Type.Exponential),
)
central_body_gravity = CentralBodyGravity(earth)
sun_third_body_gravity = ThirdBodyGravity(Sun.default())
moon_third_body_gravity = ThirdBodyGravity(Moon.default())
atmospheric_drag = AtmosphericDrag(earth, satellite_system)
constant_thrust_thruster = ConstantThrustThruster(satellite_system, thrust_direction)

In [8]:
dynamics_list = [
    central_body_gravity,
    sun_third_body_gravity,
    moon_third_body_gravity,
    atmospheric_drag,
    constant_thrust_thruster,
    PositionDerivative(),  # Required by default
]

### **Define a `NumericalSolver` (leveraging `boost::numeric::odeint`) for numerical integration with:**
* `NumericalSolver.LogType`
* `NumericalSolver.StepperType`
* `TimeStep`
* `RelativeTolerance`
* `AbsoluteTolerance`

Alternatively, you can use `NumericalSolver.default()`

In [9]:
numerical_solver = NumericalSolver(
    NumericalSolver.LogType.NoLog,
    NumericalSolver.StepperType.RungeKuttaFehlberg78,
    5.0,
    1.0e-15,
    1.0e-15,
)

### **Define a `Propagator`**

In [10]:
propagator = Propagator(numerical_solver, dynamics_list)

## Propagate

Define the propagation `instants` of interest

In [11]:
instants = Interval.closed(
    start_state.get_instant(), start_state.get_instant() + Duration.minutes(60.0)
).generate_grid(Duration.seconds(30.0))

Propagate from `start_state` for all `instants` defined

In [12]:
states = propagator.calculate_states_at(start_state, instants)

Locking local repository [/usr/local/share/open-space-toolkit-data/data]...
Fetching Data Manifest from [https://github.com/open-space-collective/open-space-toolkit-data/raw/main/data/manifest.json]...
Unlocking local repository [/usr/local/share/open-space-toolkit-data/data]...
Data  Manifest [/usr/local/share/open-space-toolkit-data/data/manifest.json] has been successfully fetched from [https://github.com/open-space-collective/open-space-toolkit-data/raw/main/data/manifest.json].


Format the output data

In [24]:
def convert_state_with_mass(
    instant: Instant, coordinates_broker: CoordinatesBroker, state: State
) -> list:
    """
    Convert an input (Instant, State) into dataframe-ready values. Including mass.
    """

    lla = LLA.cartesian(
        state.get_position()
        .in_frame(Frame.ITRF(), state.get_instant())
        .get_coordinates(),
        EarthGravitationalModel.EGM2008.equatorial_radius,
        EarthGravitationalModel.EGM2008.flattening,
    )

    return [
        repr(instant),
        float(instant.get_modified_julian_date(Scale.UTC)),
        *state.get_position().get_coordinates().transpose().tolist(),
        *state.get_velocity().get_coordinates().transpose().tolist(),
        state.extract_coordinates(CoordinatesSubset.mass())[0],
        float(lla.get_latitude().in_degrees()),
        float(lla.get_longitude().in_degrees()),
        float(lla.get_altitude().in_meters()),
    ]

In [25]:
data = [
    convert_state_with_mass(state.get_instant(), coordinates_broker, state)
    for state in states
]

Compute classical orbital elements for each state propagated

In [26]:
for i, state in enumerate(states):
    coe = COE.cartesian(
        (state.get_position(), state.get_velocity()),
        earth.get_gravitational_parameter(),
    )
    data[i] += [
        coe.get_semi_major_axis().in_kilometers(),
        coe.get_raan().in_degrees(),
    ]

In [27]:
orbit_df = pd.DataFrame(
    data,
    columns=[
        "$Time^{UTC}$",
        "$MJD^{UTC}$",
        "$x_{x}^{ECI}$",
        "$x_{y}^{ECI}$",
        "$x_{z}^{ECI}$",
        "$v_{x}^{ECI}$",
        "$v_{y}^{ECI}$",
        "$v_{z}^{ECI}$",
        "$Mass$",
        "$Latitude$",
        "$Longitude$",
        "$Altitude$",
        "$SemiMajorAxisKm$",
        "$RaanDegrees$",
    ],
)

## Display

In [28]:
orbit_df.head()

,$Time^{UTC}$,$MJD^{UTC}$,$x_{x}^{ECI}$,$x_{y}^{ECI}$,$x_{z}^{ECI}$,$v_{x}^{ECI}$,$v_{y}^{ECI}$,$v_{z}^{ECI}$,$Mass$,$Latitude$,$Longitude$,$Altitude$,$SemiMajorAxisKm$,$RaanDegrees$
0,2023-03-20 00:00:00 [UTC],60023.000000,-1.514669e+06,-192084.121491,6.831711e+06,-6348.079188,3867.582493,-1297.176104,105.000000,77.349678,10.189623,642457.261884,7000.0000000000073,330.00000076688912
1,2023-03-20 00:00:30 [UTC],60023.000347,-1.704294e+06,-75974.601253,6.789239e+06,-6292.492365,3872.381028,-1534.109550,104.979606,75.849842,5.437398,642256.719762,7000.7609353413063,330.00063910333813
2,2023-03-20 00:01:00 [UTC],60023.000694,-1.892153e+06,40218.448559,6.739681e+06,-6230.352266,3873.150538,-1769.466075,104.959211,74.277980,1.573325,642034.405171,7001.5572954774916,330.00126921694272
3,2023-03-20 00:01:30 [UTC],60023.001042,-2.078050e+06,156374.124500,6.683089e+06,-6161.722134,3869.889533,-2003.000847,104.938817,72.653524,-1.617427,641791.865389,7002.3879111260521,330.00188974555323
4,2023-03-20 00:02:00 [UTC],60023.001389,-2.261792e+06,272371.540580,6.619521e+06,-6086.671945,3862.600669,-2234.470703,104.918423,70.989814,-4.292954,641530.773182,7003.2514692529303,330.00249939391557


2D plot, over **World Map**:

In [29]:
figure = create_2d_map(
    data=go.Scattergeo(
        lon=orbit_df["$Longitude$"],
        lat=orbit_df["$Latitude$"],
        mode="lines",
        line=go.scattergeo.Line(width=1, color="red"),
    ),
)

figure.show()

3D plot, in **Earth Fixed** frame:

In [30]:
figure = create_3d_globe(
    data=[
        go.Scattergeo(
            lon=orbit_df["$Longitude$"],
            lat=orbit_df["$Latitude$"],
            mode="lines",
            line=go.scattergeo.Line(width=2, color="rgb(255, 62, 79)"),
        )
    ],
)

figure.show()

Semi-major axis evolution over time

In [31]:
figure = go.Figure()
figure.add_trace(
    go.Scatter(
        x=orbit_df["$MJD^{UTC}$"],
        y=[float(item) for item in orbit_df["$SemiMajorAxisKm$"]],
        mode="lines",
        name="Semi Major Axis (Km)",
    )
)

figure.show()

Spacecraft wet mass evolution over time

In [33]:
figure = go.Figure()
figure.add_trace(
    go.Scatter(
        x=orbit_df["$MJD^{UTC}$"],
        y=[float(item) for item in orbit_df["$Mass$"]],
        mode="lines",
        name="Spacecraft Mass (kg)",
    )
)

figure.show()